In [1]:
import more_utils
more_utils.set_logging_level("INFO")
import pyarrow
from more_utils.persistence import ModelarDB
from more_utils.time_series import ModelTable
import pyarrow as pa
from datetime import datetime

### Create connection to ModelarDB

Params:

1. hostname: ip adrress of the system where ModelarDB is running
2. interface: arrow (recommended), socket, http


In [2]:
conn_obj = ModelarDB.connect(hostname="83.212.75.52", port=31000, interface="arrow")

#### List tables in ModelarDB


In [3]:
conn_obj.list_tables()

['wind_turbine_3',
 'test_table',
 'wind_turbine',
 'wind_turbine_5',
 'wind_turbine_4',
 'wind_turbine_2']

#### Create Model Table from Parquet file: from_parquet_file()

Params:

- modelardb_conn (ModelarDB): ModelarDB connection object
- file_path (str): parquet file path


In [4]:
model_table = ModelTable.from_parquet_file(conn_obj, "/Users/dhaval/Projects/Datasets/wind_turbine.parquet")

#### Persist Table


In [5]:
model_table.persist(table_name="wind_turbine_6", error_bound=0.5)

[2023-09-14 22:34:11:842] - INFO - moreutils (ModelTable) - Model Table 'wind_turbine_6' created.
[2023-09-14 22:35:15:161] - INFO - moreutils (ModelTable) - Data inserted successfully into the table 'wind_turbine_6'.


#### Apply Flush


In [6]:
model_table.flush(flush_mode="local")

[2023-09-14 22:35:15:531] - INFO - moreutils (ModelTable) - Compressed data buffers flushed.


#### Create custom using Arrow Table format


In [7]:
# Read the data into a PyArrow table.
timestamp = pyarrow.array([
    pa.scalar(datetime(2023, 9, 10), type=pa.timestamp('us')), 
    pa.scalar(datetime(2023, 9, 12), type=pa.timestamp('us')), 
    pa.scalar(datetime(2023, 9, 14), type=pa.timestamp('us'))])
wind_turbine = pyarrow.array(["1026", "1026", "1026"])
wind_direction = pyarrow.array([300.0, 300.0, 300.0])
wind_speed = pyarrow.array([4.0, 4.0, 4.0])

names = ["timestamp", "wind_turbine", "wind_direction", "wind_speed"]
table = pyarrow.Table.from_arrays([timestamp, wind_turbine, wind_direction, wind_speed], names=names)

#### Create Model Table from Parquet file: from_arrow_table()

Params:

- modelardb_conn (ModelarDB): ModelarDB connection object
- arrow_table (str): pyarrow table format

In [ ]:
model_table = ModelTable.from_arrow_table(conn_obj, table)

#### Persist and flush


In [8]:
model_table.persist(table_name="test_table_2", error_bound=0.5)
model_table.flush(flush_mode="local")

[2023-09-14 22:35:16:182] - INFO - moreutils (ModelTable) - Model Table 'test_table_2' created.
[2023-09-14 22:35:16:400] - INFO - moreutils (ModelTable) - Data inserted successfully into the table 'test_table_2'.
[2023-09-14 22:35:16:609] - INFO - moreutils (ModelTable) - Compressed data buffers flushed.
